# Practice notebook for hypothesis tests using NHANES data

This notebook will give you the opportunity to perform some hypothesis tests with the NHANES data that are similar to
what was done in the week 3 case study notebook.

You can enter your code into the cells that say "enter your code here", and you can type responses to the questions into the cells that say "Type Markdown and Latex".

Note that most of the code that you will need to write below is very similar to code that appears in the case study notebook.  You will need to edit code from that notebook in small ways to adapt it to the prompts below.

To get started, we will use the same module imports and read the data in the same way as we did in the case study:

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import statsmodels.stats.proportion as smprop
import numpy as np
import scipy.stats.distributions as dist


In [2]:

da = pd.read_csv("nhanes_2015_2016.csv")

## Question 1

Conduct a hypothesis test (at the 0.05 level) for the null hypothesis that the proportion of women who smoke is equal to the proportion of men who smoke.

In [3]:
# insert your code here
da["SMQ020x"] = da.SMQ020.replace({1: "Yes", 2: "No", 7: np.nan, 9: np.nan})
da["RIAGENDRx"] = da.RIAGENDR.replace({1: "Male", 2: "Female"})
dx = da[["SMQ020x", "RIDAGEYR", "RIAGENDRx"]].dropna()

dx.head()

,SMQ020x,RIDAGEYR,RIAGENDRx
0,Yes,62,Male
1,Yes,53,Male
2,Yes,78,Male
3,No,56,Female
4,No,42,Female


In [4]:
p = dx.groupby("RIAGENDRx")["SMQ020x"].agg([lambda z: np.mean(z == "Yes"), "size"])
p.columns = ["Smoke", "N"]
print(p)

              Smoke     N
RIAGENDRx                
Female     0.304845  2972
Male       0.513258  2753


In [12]:
da["SMQ020xf"] = da.SMQ020x.replace({"Yes":1, "No":0})
females = da[da["RIAGENDRx"] == 'Female']['SMQ020xf']
males = da[da["RIAGENDRx"] == 'Male']['SMQ020xf']


0       1.0
1       1.0
2       1.0
6       1.0
8       0.0
       ... 
5726    1.0
5728    0.0
5729    0.0
5731    0.0
5733    1.0
Name: SMQ020xf, Length: 2759, dtype: float64

In [13]:
sm.stats.ttest_ind(females,males)


(nan, nan, 5733.0)

In [5]:
p_comb = (dx.SMQ020x == "Yes").mean()
va = p_comb * (1 - p_comb)

se = np.sqrt(va * (1 / p.N.Female + 1 / p.N.Male))

In [6]:
(p_comb, va, se)

(0.4050655021834061, 0.2409874411243111, 0.01298546309757376)

In [7]:
test_stat = (p.Smoke.Female - p.Smoke.Male) / se
p_value = 2 * dist.norm.cdf(-np.abs(test_stat))
(test_stat, p_value)

(-16.049719603652488, 5.742288777302776e-58)

__Q1a.__ Write 1-2 sentences explaining the substance of your findings to someone who does not know anything about statistical hypothesis tests.

what we have here p_value = 5.74e-58 
represent the propability of the difference between males and females is zero
accourding to our sample data which is extremely low ( reject the null hypothesis )
which means there is a difference between males and females in smoking 

__Q1b.__ Construct three 95% confidence intervals: one for the proportion of women who smoke, one for the proportion of men who smoke, and one for the difference in the rates of smoking between women and men.

### Using Stats model

In [12]:
# 95% CI for the proportion of females who smoke  
sm.stats.proportion_confint( 0.304845  * 2972, 2972)  

(0.28829476929812486, 0.3213952307018751)

In [13]:
# 95% CI for the proportion of Males who smoke 
sm.stats.proportion_confint( 0.513258 * 2753, 2753)

(0.4945872286635714, 0.5319287713364286)

In [16]:
smprop.confint_proportions_2indep( 0.304845  * 2972, 2972, 0.513258 * 2753, 2753, compare='diff', alpha=0.05, correction=True)

(-0.2331672407090571, -0.18329687359405544)

### Using Numpy model

In [8]:
# insert your code here
p = .304845
n = 2972
se_female = np.sqrt(p * (1 - p)/n)
se_female

0.00844415041930423

In [18]:
# CI for females
p = .304845 
lcb = p - 1.96 * se_female
ucb = p + 1.96 * se_female
(lcb, ucb)

(0.2882944651781637, 0.32139553482183625)

In [9]:
p = .513258
n = 2753
se_male = np.sqrt(p * (1 - p)/ n)
se_male

0.009526078787008965

In [19]:
# CI for Males
p = .513258 
lcb = p - 1.96 * se_male
ucb = p + 1.96 * se_male
(lcb, ucb)

(0.49458688557746244, 0.5319291144225375)

In [10]:
se_diff = np.sqrt(se_female**2 + se_male**2)
se_diff

0.012729880335656654

In [11]:
d = .304845 - .513258
lcb = d - 1.96 * se_diff
ucb = d + 1.96 * se_diff
(lcb, ucb)

(-0.23336356545788706, -0.18346243454211297)

__Q1c.__ Comment on any ways in which the confidence intervals that you found in part b reinforce, contradict, or add support to the hypothesis test conducted in part a.

there is no differece they both conferm the difference
but Confidence interval add more support countify the dicsion  

## Question 2

Partition the population into two groups based on whether a person has graduated college or not, using the educational attainment variable [DMDEDUC2](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#DMDEDUC2).  Then conduct a test of the null hypothesis that the average heights (in centimeters) of the two groups are equal.  Next, convert the heights from centimeters to inches, and conduct a test of the null hypothesis that the average heights (in inches) of the two groups are equal.

In [ ]:
# insert your code here

__Q2a.__ Based on the analysis performed here, are you confident that people who graduated from college have a different average height compared to people who did not graduate from college?

__Q2b:__ How do the results obtained using the heights expressed in inches compare to the results obtained using the heights expressed in centimeters?

## Question 3

Conduct a hypothesis test of the null hypothesis that the average BMI for men between 30 and 40 is equal to the average BMI for men between 50 and 60.  Then carry out this test again after log transforming the BMI values.

In [ ]:
# insert your code here

__Q3a.__ How would you characterize the evidence that mean BMI differs between these age bands, and how would you characterize the evidence that mean log BMI differs between these age bands?

## Question 4

Suppose we wish to compare the mean BMI between college graduates and people who have not graduated from college, focusing on women between the ages of 30 and 40.  First, consider the variance of BMI within each of these subpopulations using graphical techniques, and through the estimated subpopulation variances.  Then, calculate pooled and unpooled estimates of the standard error for the difference between the mean BMI in the two populations being compared.  Finally, test the null hypothesis that the two population means are equal, using each of the two different standard errors.

In [ ]:
# insert your code here

__Q4a.__ Comment on the strength of evidence against the null hypothesis that these two populations have equal mean BMI.

__Q4b.__ Comment on the degree to which the two populations have different variances, and on the extent to which the results using different approaches to estimating the standard error of the mean difference give divergent results.

## Question 5

Conduct a test of the null hypothesis that the first and second diastolic blood pressure measurements within a subject have the same mean values.

In [ ]:
# insert your code here

__Q5a.__ Briefly describe your findings for an audience that is not familiar with statistical hypothesis testing.

__Q5b.__ Pretend that the first and second diastolic blood pressure measurements were taken on different people.  Modfify the analysis above as appropriate for this setting.

In [ ]:
# insert your code here

__Q5c.__ Briefly describe how the approaches used and the results obtained in the preceeding two parts of the question differ.